In [6]:
import pandas as pd
import requests
import json

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [7]:
sns.set(
    font_scale =2,
    style      ='darkgrid',
    rc         ={'figure.figsize': (20, 7)}
)

### Работа с Яндекс.Метрикой

In [8]:
# Base url to service
url = 'https://api-metrika.yandex.net/stat/v1/data?'

# &-separated parameters of query in a form of name=value, taken from the metrica site
visits = 'metrics=ym:s:visits&dimensions=ym:s:date&&dimensions=ym:s:isRobot&id=44147844'
url = url + visits
   
# Get json of response
query = requests.get(url)
json_data = json.loads(query.text)
   
# Conversion of obtained json to dataframe
visits_data = pd.DataFrame([(
                            i['dimensions'][0]['name'],
                            i['metrics'][0], 
                            i['dimensions'][1]['name']) for i in json_data['data']],                            
                            columns=['date', 'visits', 'user_type'])


In [9]:
visits_by_day = visits_data.groupby(['date', 'user_type'], as_index=False).agg({'visits': 'sum'})

In [10]:
px.bar(data_frame=visits_by_day, y='visits', x='date', color='user_type', barmode='group', title='Visits by date')

In [11]:
url = 'https://api-metrika.yandex.net/stat/v1/data?'

# &-separated parameters of query in a form of name=value, taken from the metrica site
visits = "metrics=ym:s:visits&dimensions=ym:s:date,ym:s:isRobot,ym:s:browser&id=44147844"
url = url + visits
   
# Get json of response
query = requests.get(url)
json_data = json.loads(query.text)
   
# Conversion of obtained json to dataframe
visits_data = pd.DataFrame([(
                            i['dimensions'][0]['name'],
                            i['metrics'][0], 
                            i['dimensions'][1]['name'], 
                            i['dimensions'][2]['name']) for i in json_data['data']],
                            columns=['date', 'visits', 'user_type', 'browser'])

In [12]:
visits_data.head(3)

,date,visits,user_type,browser
0,2021-05-05,157.0,People,Google Chrome
1,2021-05-04,153.0,People,Google Chrome
2,2021-04-30,142.0,People,Google Chrome


In [13]:
(visits_data.query('browser == "Google Chrome"').visits.sum() / visits_data.visits.sum() * 100).round()

57.0

In [14]:
data_grouped = visits_data.groupby('browser', as_index=False).agg('sum').sort_values('visits',ascending=False)
px.bar(data_frame=data_grouped, y='visits', x='browser', barmode='group', title='Visits by browser')

In [15]:
visits_by_browser = visits_data.groupby(['browser', 'user_type'], as_index=False) \
            .agg({'visits': 'sum'}) \
            .pivot(index='browser', columns='user_type', values='visits').reset_index().fillna(0)

In [16]:
visits_by_browser['robots_perc'] = visits_by_browser.Robots / (visits_by_browser.Robots + visits_by_browser.People)  * 100
visits_by_browser['people_perc'] = visits_by_browser.People / (visits_by_browser.Robots + visits_by_browser.People)  * 100

In [17]:
visits_by_browser = visits_by_browser.sort_values('robots_perc', ascending=False)

In [18]:
px.bar(data_frame=visits_by_browser, x='browser', y='robots_perc', labels={'robots_perc': 'Robots, %'}, title='How much users of each browser are robots')

### Работа с Google таблицами

In [19]:
from io import BytesIO
import requests
r = requests.get('https://docs.google.com/spreadsheets/d/e/2PACX-1vTOoCXy-lo0ce_pyrjDR1YsDD-o6Uxmiwqiwj5vm7n6i_WJFrf2juovZf16bIm6TjCD470u_j2HXRPn/pub?output=csv')
data = r.content

df = pd.read_csv(BytesIO(data)).drop('date', axis=1) 

In [20]:
visits_by_browser = visits_data.groupby(['browser', 'user_type'], as_index=False) \
            .agg({'visits': 'sum'}) \
            .pivot(index='browser', columns='user_type', values='visits').reset_index()

In [21]:
df_by_browser = df.groupby(['browser', 'user_type'], as_index=False) \
            .agg({'users': 'sum'}) \
            .pivot(index='browser', columns='user_type', values='users').reset_index()

Объединяем данные, полученные из Яндекс.Метрики и Google таблиц

In [22]:
all_data = visits_by_browser.merge(df_by_browser, on='browser')

In [23]:
all_data['People'] = all_data.People_x + all_data.People_y
all_data['Robots'] = all_data.Robots_x + all_data.Robots_y

In [24]:
all_data = all_data[['browser', 'People', 'Robots']]

In [25]:
all_data.head(3)

user_type,browser,People,Robots
0,Android Browser,48.0,3.0
1,Chrome Mobile,278.0,3.0
2,Edge,39.0,34.0
